In [19]:
import torch 
from torch.nn import ReLU , Conv2d,Dropout,Flatten, MaxPool2d,Flatten,Linear,CrossEntropyLoss
import numpy as np
import torch.optim as optim
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

batch_size = 32
tranform  = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_set = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=tranform)
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True)
#32*32 images . 60K
#6000 images per class
test_set = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=tranform)
test_loader = DataLoader(test_set,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
len(train_set[0][0][0])

32

In [20]:
class Cifar10(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(3,32,kernel_size=(3,3),stride=1,padding=1)
        self.act1 = ReLU()
        self.drop1 = Dropout(0.2)

        self.conv2 = Conv2d(32,32,kernel_size=(3,3),stride=1,padding=1)
        self.act2 = ReLU()
        self.pool2 = MaxPool2d(kernel_size=(2,2))

        self.flat = Flatten()
        self.fc3 = Linear(8192,512)
        self.act3 = ReLU()
        self.drop2 = Dropout(0.2)

        self.fc4 = Linear(512,10)

    def forward(self,x):
        x=self.act1(self.conv1(x))
        x=self.drop1(x)

        x=self.act2(self.conv2(x))
        x=self.pool2(x)

        x=self.flat(x)
        x=self.act3(self.fc3(x))
        x=self.drop2(x)
        
        x=self.fc4(x)
        return x
model = Cifar10()
loss_fc = CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(),lr=0.01)


In [23]:

epochs = 10
epoch_loss =[]
for epoch in range(epochs):
    train_loss =0
    for i, data in enumerate(train_loader):
        input,label = data
        optimizer.zero_grad()
        output = model(input)
        loss = loss_fc(output,label)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
        if i%500 == 499:
            print('Epoch:',epoch,' batch: ',i ,' Loss:',train_loss)

    
    print('Epoch:',epoch,' Loss',train_loss)
    epoch_loss.append(train_loss)


Epoch: 0  batch:  499  Loss: 1117.289217710495
Epoch: 0  batch:  999  Loss: 2214.0124990940094
Epoch: 0  batch:  1499  Loss: 3289.148059487343
Epoch: 0  Loss 3423.3773118257523
Epoch: 1  batch:  499  Loss: 1053.1954149007797
Epoch: 1  batch:  999  Loss: 2087.0221832990646
Epoch: 1  batch:  1499  Loss: 3094.537964463234
Epoch: 1  Loss 3220.172474384308
Epoch: 2  batch:  499  Loss: 989.5647174119949
Epoch: 2  batch:  999  Loss: 1968.4214589595795
Epoch: 2  batch:  1499  Loss: 2931.1206098794937
Epoch: 2  Loss 3051.698788523674
Epoch: 3  batch:  499  Loss: 954.6911479234695
Epoch: 3  batch:  999  Loss: 1904.2924333810806
Epoch: 3  batch:  1499  Loss: 2846.773894071579
Epoch: 3  Loss 2964.449272632599
Epoch: 4  batch:  499  Loss: 933.977147102356
Epoch: 4  batch:  999  Loss: 1857.1621400117874
Epoch: 4  batch:  1499  Loss: 2776.740075469017
Epoch: 4  Loss 2891.4179512262344
Epoch: 5  batch:  499  Loss: 909.5954811573029
Epoch: 5  batch:  999  Loss: 1812.9063835144043
Epoch: 5  batch:  1499

In [24]:
model.eval()
correct =0
total =0
with torch.no_grad():  # Disable gradient calculation for inference
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {(correct / total) * 100:.2f}%')

Test Accuracy: 43.90%
